## 1. Import Libraries


In [ ]:
# Standard libraries
import os
import sys
import json
import time
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any

# Data handling
import numpy as np
import pandas as pd

# Image processing
from PIL import Image
import cv2

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Progress tracking
from tqdm.auto import tqdm

# Deep learning
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.transforms.functional as TF

# Configuration
import yaml

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("✅ Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Setup Paths and Load Configuration

In [ ]:
# Project paths
project_root = Path(r'd:\Projects\AI-Virtual-TryOn')
dataset_root = project_root / 'data' / 'zalando-hd-resized'
processed_dir = project_root / 'data' / 'processed'
output_dir = project_root / 'outputs' / 'dataset'
output_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Project Root: {project_root}")
print(f"📁 Dataset Root: {dataset_root}")
print(f"📁 Processed Data: {processed_dir}")
print(f"📁 Output Directory: {output_dir}")

# Load configurations
with open(processed_dir / 'preprocessing_config.yaml', 'r') as f:
    preprocess_config = yaml.load(f, Loader=yaml.FullLoader)

with open(project_root / 'outputs' / 'cloth_agnostic' / 'cloth_agnostic_config.json', 'r') as f:
    ca_config = json.load(f)

print("\n✅ Loaded configurations")

# Device configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\n🖥️ Using device: {device}")

# Load dataset catalogs
train_catalog = pd.read_csv(processed_dir / 'train_catalog.csv')
val_catalog = pd.read_csv(processed_dir / 'val_catalog.csv')
test_catalog = pd.read_csv(processed_dir / 'test_catalog.csv')

print(f"\n📊 Dataset: {len(train_catalog)} train, {len(val_catalog)} val, {len(test_catalog)} test")

## 3. Define Helper Functions

Reuse functions from previous notebooks.

In [ ]:
def load_pose_keypoints(json_path: str) -> np.ndarray:
    """Load pose keypoints from OpenPose JSON."""
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    if 'people' in data and len(data['people']) > 0:
        pose_keypoints = data['people'][0]['pose_keypoints_2d']
        keypoints = np.array(pose_keypoints).reshape(-1, 3)
        return keypoints
    else:
        return np.zeros((25, 3))


def create_pose_heatmaps(keypoints: np.ndarray, 
                        image_size: Tuple[int, int],
                        sigma: float = 3.0) -> np.ndarray:
    """Create Gaussian heatmaps from keypoints."""
    h, w = image_size
    heatmaps = np.zeros((18, h, w), dtype=np.float32)  # 18 channels
    
    y_grid, x_grid = np.ogrid[:h, :w]
    
    for i in range(18):  # Only first 18 keypoints
        x, y, conf = keypoints[i]
        if conf > 0.1:
            x, y = int(x), int(y)
            if 0 <= x < w and 0 <= y < h:
                heatmap = np.exp(-((x_grid - x)**2 + (y_grid - y)**2) / (2 * sigma**2))
                heatmaps[i] = heatmap * conf
    
    return heatmaps


def parsing_to_onehot(parsing: np.ndarray, num_classes: int = 20) -> np.ndarray:
    """Convert parsing mask to one-hot encoding."""
    h, w = parsing.shape
    onehot = np.zeros((num_classes, h, w), dtype=np.float32)
    
    for i in range(num_classes):
        onehot[i][parsing == i] = 1.0
    
    return onehot


def create_cloth_agnostic_mask(parsing: np.ndarray,
                               garment_classes: List[int] = [5, 7]) -> np.ndarray:
    """Create cloth-agnostic mask by removing garment."""
    mask = np.ones_like(parsing, dtype=np.uint8)
    
    for cls_id in garment_classes:
        mask[parsing == cls_id] = 0
    
    return mask


print("✅ Helper functions defined")

## 4. Data Augmentation Pipeline

In [ ]:
class VITONAugmentation:
    """
    Data augmentation for virtual try-on.
    Applies synchronized augmentation to all modalities.
    """
    
    def __init__(self, 
                 is_train: bool = True,
                 horizontal_flip_prob: float = 0.5,
                 color_jitter_prob: float = 0.3):
        self.is_train = is_train
        self.horizontal_flip_prob = horizontal_flip_prob
        self.color_jitter_prob = color_jitter_prob
        
        # Color jitter for RGB images only
        self.color_jitter = T.ColorJitter(
            brightness=0.2,
            contrast=0.2,
            saturation=0.2,
            hue=0.1
        )
    
    def __call__(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Apply augmentation to all data modalities.
        """
        if not self.is_train:
            return data
        
        # Horizontal flip (synchronized across all modalities)
        if torch.rand(1).item() < self.horizontal_flip_prob:
            data = self._horizontal_flip(data)
        
        # Color jitter (only for RGB images)
        if torch.rand(1).item() < self.color_jitter_prob:
            data = self._apply_color_jitter(data)
        
        return data
    
    def _horizontal_flip(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """Apply horizontal flip to all modalities."""
        # Flip image tensors
        for key in ['person_image', 'cloth_image', 'cloth_agnostic', 
                    'pose_map', 'parsing_onehot', 'ca_mask']:
            if key in data:
                data[key] = TF.hflip(data[key])
        
        # Swap left/right keypoints if needed
        # This would require more complex logic for pose keypoints
        
        return data
    
    def _apply_color_jitter(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """Apply color jitter to RGB images only."""
        for key in ['person_image', 'cloth_image']:
            if key in data:
                data[key] = self.color_jitter(data[key])
        
        return data


print("✅ Augmentation pipeline defined")

## 5. VITONDataset Class

In [ ]:
class VITONDataset(Dataset):
    """
    PyTorch Dataset for Virtual Try-On (VITON-HD).
    
    Returns:
    - person_image: [3, H, W] normalized RGB
    - cloth_image: [3, H, W] normalized RGB
    - cloth_agnostic: [3, H, W] normalized RGB (with garment removed)
    - pose_map: [18, H, W] pose heatmaps
    - parsing_onehot: [20, H, W] parsing one-hot
    - ca_mask: [1, H, W] cloth-agnostic mask
    """
    
    def __init__(self,
                 catalog: pd.DataFrame,
                 image_size: Tuple[int, int] = (1024, 768),
                 is_train: bool = True,
                 augmentation: Optional[VITONAugmentation] = None):
        """
        Args:
            catalog: DataFrame with file paths
            image_size: (height, width)
            is_train: Whether this is training set
            augmentation: Augmentation pipeline
        """
        self.catalog = catalog.reset_index(drop=True)
        self.image_size = image_size
        self.is_train = is_train
        self.augmentation = augmentation
        
        # Normalization parameters
        self.mean = torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1)
        self.std = torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1)
        
        # Garment classes to remove for cloth-agnostic
        self.upper_body_classes = [5, 7]  # Upper-clothes, Coat
    
    def __len__(self) -> int:
        return len(self.catalog)
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        """
        Load and preprocess a single sample.
        """
        row = self.catalog.iloc[idx]
        
        # Load images
        person_img = np.array(Image.open(row['person_image']))
        cloth_img = np.array(Image.open(row['cloth_image']))
        parsing = np.array(Image.open(row['parse_mask']))
        keypoints = load_pose_keypoints(row['pose_json'])
        
        # Create cloth-agnostic RGB
        ca_mask = create_cloth_agnostic_mask(parsing, self.upper_body_classes)
        ca_rgb = person_img.copy()
        ca_rgb[ca_mask == 0] = 128  # Gray fill
        
        # Create pose heatmaps
        pose_heatmaps = create_pose_heatmaps(keypoints, self.image_size)
        
        # Create parsing one-hot
        parsing_onehot = parsing_to_onehot(parsing, num_classes=20)
        
        # Convert to tensors
        person_tensor = torch.from_numpy(person_img).permute(2, 0, 1).float() / 255.0
        cloth_tensor = torch.from_numpy(cloth_img).permute(2, 0, 1).float() / 255.0
        ca_tensor = torch.from_numpy(ca_rgb).permute(2, 0, 1).float() / 255.0
        pose_tensor = torch.from_numpy(pose_heatmaps).float()
        parsing_tensor = torch.from_numpy(parsing_onehot).float()
        ca_mask_tensor = torch.from_numpy(ca_mask).unsqueeze(0).float()
        
        # Normalize RGB images to [-1, 1]
        person_tensor = (person_tensor - self.mean) / self.std
        cloth_tensor = (cloth_tensor - self.mean) / self.std
        ca_tensor = (ca_tensor - self.mean) / self.std
        
        # Create data dictionary
        data = {
            'person_image': person_tensor,        # [3, H, W]
            'cloth_image': cloth_tensor,          # [3, H, W]
            'cloth_agnostic': ca_tensor,          # [3, H, W]
            'pose_map': pose_tensor,              # [18, H, W]
            'parsing_onehot': parsing_tensor,     # [20, H, W]
            'ca_mask': ca_mask_tensor,            # [1, H, W]
            'id': row['id']
        }
        
        # Apply augmentation
        if self.augmentation is not None:
            data = self.augmentation(data)
        
        return data


print("✅ VITONDataset class defined")

## 6. Create DataLoaders

In [ ]:
# Create augmentation pipelines
train_augmentation = VITONAugmentation(
    is_train=True,
    horizontal_flip_prob=0.5,
    color_jitter_prob=0.3
)

val_augmentation = VITONAugmentation(
    is_train=False
)

# Create datasets
train_dataset = VITONDataset(
    catalog=train_catalog,
    image_size=(1024, 768),
    is_train=True,
    augmentation=train_augmentation
)

val_dataset = VITONDataset(
    catalog=val_catalog,
    image_size=(1024, 768),
    is_train=False,
    augmentation=val_augmentation
)

test_dataset = VITONDataset(
    catalog=test_catalog,
    image_size=(1024, 768),
    is_train=False,
    augmentation=val_augmentation
)

print("✅ Datasets created")
print(f"   Train: {len(train_dataset):,} samples")
print(f"   Val: {len(val_dataset):,} samples")
print(f"   Test: {len(test_dataset):,} samples")

# Create DataLoaders
# Note: num_workers=0 for Windows compatibility
batch_size = 4

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,  # Windows compatibility
    pin_memory=True if torch.cuda.is_available() else False,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False,
    drop_last=False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False,
    drop_last=False
)

print("\n✅ DataLoaders created")
print(f"   Train: {len(train_loader):,} batches")
print(f"   Val: {len(val_loader):,} batches")
print(f"   Test: {len(test_loader):,} batches")
print(f"   Batch size: {batch_size}")

## 7. Test Single Sample

In [ ]:
# Test loading a single sample
sample = train_dataset[0]

print("="*70)
print("🔍 SINGLE SAMPLE TEST")
print("="*70)

print(f"\n📊 Sample ID: {sample['id']}")
print(f"\n📊 Data shapes:")
for key, value in sample.items():
    if isinstance(value, torch.Tensor):
        print(f"   {key:25s}: {tuple(value.shape)}")
        print(f"      dtype: {value.dtype}, range: [{value.min():.3f}, {value.max():.3f}]")

# Calculate total channels
total_channels = (
    sample['cloth_agnostic'].shape[0] +  # 3
    sample['pose_map'].shape[0] +         # 18
    sample['parsing_onehot'].shape[0]     # 20
)

print(f"\n💡 Total model input channels: {total_channels}")
print(f"   - Cloth-agnostic RGB: {sample['cloth_agnostic'].shape[0]}")
print(f"   - Pose map: {sample['pose_map'].shape[0]}")
print(f"   - Parsing one-hot: {sample['parsing_onehot'].shape[0]}")

print("\n" + "="*70)
print("\n✅ Single sample loaded successfully!")

## 8. Test Batch Loading

In [ ]:
# Load a batch
print("="*70)
print("📦 BATCH LOADING TEST")
print("="*70)

batch = next(iter(train_loader))

print(f"\n📊 Batch shapes:")
for key, value in batch.items():
    if isinstance(value, torch.Tensor):
        print(f"   {key:25s}: {tuple(value.shape)}")
        print(f"      dtype: {value.dtype}, device: {value.device}")

print(f"\n📊 Memory usage:")
total_memory = 0
for key, value in batch.items():
    if isinstance(value, torch.Tensor):
        mem = value.element_size() * value.nelement() / 1024**2  # MB
        total_memory += mem
        print(f"   {key:25s}: {mem:.2f} MB")

print(f"   {'Total':25s}: {total_memory:.2f} MB")

print("\n" + "="*70)
print("\n✅ Batch loaded successfully!")

## 9. Visualize Batch

In [ ]:
def denormalize(tensor: torch.Tensor) -> torch.Tensor:
    """Denormalize from [-1, 1] to [0, 1]."""
    return tensor * 0.5 + 0.5


# Visualize batch
fig, axes = plt.subplots(4, 5, figsize=(20, 16))

for i in range(4):  # Show 4 samples from batch
    # Person image
    person = denormalize(batch['person_image'][i]).permute(1, 2, 0).cpu().numpy()
    axes[i, 0].imshow(np.clip(person, 0, 1))
    axes[i, 0].set_title(f"Person\n{batch['id'][i]}" if i == 0 else batch['id'][i], fontsize=10)
    axes[i, 0].axis('off')
    
    # Cloth image
    cloth = denormalize(batch['cloth_image'][i]).permute(1, 2, 0).cpu().numpy()
    axes[i, 1].imshow(np.clip(cloth, 0, 1))
    axes[i, 1].set_title('Cloth' if i == 0 else '', fontsize=10)
    axes[i, 1].axis('off')
    
    # Cloth-agnostic
    ca = denormalize(batch['cloth_agnostic'][i]).permute(1, 2, 0).cpu().numpy()
    axes[i, 2].imshow(np.clip(ca, 0, 1))
    axes[i, 2].set_title('Cloth-Agnostic' if i == 0 else '', fontsize=10)
    axes[i, 2].axis('off')
    
    # Pose map (max across channels)
    pose = torch.max(batch['pose_map'][i], dim=0)[0].cpu().numpy()
    axes[i, 3].imshow(pose, cmap='hot')
    axes[i, 3].set_title('Pose Map' if i == 0 else '', fontsize=10)
    axes[i, 3].axis('off')
    
    # Parsing (argmax to get class labels)
    parsing = torch.argmax(batch['parsing_onehot'][i], dim=0).cpu().numpy()
    axes[i, 4].imshow(parsing, cmap='tab20')
    axes[i, 4].set_title('Parsing' if i == 0 else '', fontsize=10)
    axes[i, 4].axis('off')

plt.suptitle(f'Batch Visualization (Batch size: {batch_size})', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(output_dir / 'batch_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Batch visualization saved")

## 10. Timing Benchmark

In [ ]:
print("="*70)
print("⏱️ DATALOADER TIMING BENCHMARK")
print("="*70)

# Benchmark loading speed
n_batches = 20
times = []

print(f"\n🔄 Loading {n_batches} batches...\n")

for i, batch in enumerate(tqdm(train_loader, total=n_batches, desc="Timing")):
    start = time.time()
    
    # Simulate moving to GPU
    if torch.cuda.is_available():
        for key in batch:
            if isinstance(batch[key], torch.Tensor):
                batch[key] = batch[key].to(device)
    
    times.append(time.time() - start)
    
    if i >= n_batches - 1:
        break

times = np.array(times)

print(f"\n📊 Timing Statistics:")
print("-" * 50)
print(f"   Mean time per batch: {times.mean()*1000:.2f} ms")
print(f"   Std time per batch: {times.std()*1000:.2f} ms")
print(f"   Min time per batch: {times.min()*1000:.2f} ms")
print(f"   Max time per batch: {times.max()*1000:.2f} ms")

# Calculate samples per second
samples_per_second = batch_size / times.mean()
print(f"\n⚡ Throughput: {samples_per_second:.2f} samples/second")

# Estimate epoch time
epoch_time = len(train_loader) * times.mean() / 60  # minutes
print(f"📊 Estimated epoch time: {epoch_time:.2f} minutes")

print("\n" + "="*70)
print("\n✅ Timing benchmark complete!")

## 11. Memory Analysis

In [ ]:
print("="*70)
print("💾 MEMORY ANALYSIS")
print("="*70)

# Single sample memory
sample = train_dataset[0]
sample_memory = 0

print(f"\n📊 Single Sample Memory:")
print("-" * 50)
for key, value in sample.items():
    if isinstance(value, torch.Tensor):
        mem = value.element_size() * value.nelement() / 1024**2
        sample_memory += mem
        print(f"   {key:25s}: {mem:.2f} MB")

print(f"   {'Total':25s}: {sample_memory:.2f} MB")

# Batch memory
batch = next(iter(train_loader))
batch_memory = 0

print(f"\n📊 Batch Memory (batch_size={batch_size}):")
print("-" * 50)
for key, value in batch.items():
    if isinstance(value, torch.Tensor):
        mem = value.element_size() * value.nelement() / 1024**2
        batch_memory += mem
        print(f"   {key:25s}: {mem:.2f} MB")

print(f"   {'Total':25s}: {batch_memory:.2f} MB")

# GPU memory if available
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    
    # Move batch to GPU
    for key in batch:
        if isinstance(batch[key], torch.Tensor):
            batch[key] = batch[key].to(device)
    
    gpu_memory = torch.cuda.memory_allocated() / 1024**2
    gpu_reserved = torch.cuda.memory_reserved() / 1024**2
    
    print(f"\n📊 GPU Memory:")
    print("-" * 50)
    print(f"   Allocated: {gpu_memory:.2f} MB")
    print(f"   Reserved: {gpu_reserved:.2f} MB")
    print(f"   Total GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**2:.2f} MB")

print("\n" + "="*70)
print("\n✅ Memory analysis complete!")

## 12. Save Dataset Configuration

In [ ]:
# Create dataset configuration
dataset_config = {
    'dataset': {
        'train_samples': len(train_dataset),
        'val_samples': len(val_dataset),
        'test_samples': len(test_dataset),
        'total_samples': len(train_dataset) + len(val_dataset) + len(test_dataset)
    },
    'dataloader': {
        'batch_size': batch_size,
        'num_workers': 0,
        'pin_memory': torch.cuda.is_available(),
        'train_batches': len(train_loader),
        'val_batches': len(val_loader),
        'test_batches': len(test_loader)
    },
    'data_format': {
        'image_size': list(train_dataset.image_size),
        'channels': {
            'person_image': 3,
            'cloth_image': 3,
            'cloth_agnostic': 3,
            'pose_map': 18,
            'parsing_onehot': 20,
            'ca_mask': 1,
            'total_input': 41
        },
        'normalization': {
            'mean': [0.5, 0.5, 0.5],
            'std': [0.5, 0.5, 0.5],
            'range': [-1, 1]
        }
    },
    'augmentation': {
        'horizontal_flip_prob': 0.5,
        'color_jitter_prob': 0.3
    },
    'performance': {
        'mean_batch_time_ms': float(times.mean() * 1000),
        'throughput_samples_per_sec': float(samples_per_second),
        'estimated_epoch_time_min': float(epoch_time),
        'single_sample_memory_mb': float(sample_memory),
        'batch_memory_mb': float(batch_memory)
    }
}

# Save configuration
config_path = output_dir / 'dataset_config.json'
with open(config_path, 'w') as f:
    json.dump(dataset_config, f, indent=2)

print("="*70)
print("💾 DATASET CONFIGURATION SAVED")
print("="*70)

print(f"\n📄 Config saved to: {config_path}")

print(f"\n📊 Summary:")
print(f"   - Total samples: {dataset_config['dataset']['total_samples']:,}")
print(f"   - Batch size: {dataset_config['dataloader']['batch_size']}")
print(f"   - Input channels: {dataset_config['data_format']['channels']['total_input']}")
print(f"   - Throughput: {dataset_config['performance']['throughput_samples_per_sec']:.2f} samples/sec")
print(f"   - Epoch time: {dataset_config['performance']['estimated_epoch_time_min']:.2f} min")

print("\n" + "="*70)

## 13. Summary

In [ ]:
print("="*70)
print("🎉 PYTORCH DATASET & DATALOADER COMPLETE!")
print("="*70)

print("\n✅ Completed Tasks:")
print("   1. ✓ Implemented VITONDataset class")
print("   2. ✓ Created data augmentation pipeline")
print("   3. ✓ Built training, validation, and test DataLoaders")
print("   4. ✓ Tested single sample and batch loading")
print("   5. ✓ Benchmarked loading performance")
print("   6. ✓ Analyzed memory usage")
print("   7. ✓ Saved dataset configuration")

print(f"\n📊 Key Metrics:")
print(f"   - Train samples: {len(train_dataset):,}")
print(f"   - Val samples: {len(val_dataset):,}")
print(f"   - Test samples: {len(test_dataset):,}")
print(f"   - Batch size: {batch_size}")
print(f"   - Input channels: 41 (3 + 18 + 20)")
print(f"   - Throughput: {samples_per_second:.2f} samples/sec")
print(f"   - Epoch time: {epoch_time:.2f} minutes")

print(f"\n📁 Generated Files:")
print(f"   - batch_visualization.png")
print(f"   - dataset_config.json")

print("\n🚀 Ready for Next Steps:")
print("   1. Begin model architecture development")
print("   2. Implement Generator network")
print("   3. Implement Discriminator network")
print("   4. Create training loop with losses")
print("   5. Start model training")

print("\n💡 Key Components Created:")
print("   - VITONDataset: Complete PyTorch Dataset")
print("     * Load all modalities (person, cloth, parsing, pose)")
print("     * Generate cloth-agnostic representation")
print("     * Create multi-channel input (41 channels)")
print("     * Support data augmentation")
print("   - VITONAugmentation: Synchronized augmentation")
print("     * Horizontal flip (50% probability)")
print("     * Color jitter (30% probability)")
print("   - DataLoaders: Optimized batch loading")
print("     * Windows compatible (num_workers=0)")
print("     * GPU memory pinning enabled")
print("     * Efficient batch processing")

print("\n" + "="*70)

print("\n✅ DATASET MODULE READY FOR TRAINING!")
print("="*70)

# Model Architecture


In [ ]:
# Ensure `torchsummary` is installed and importable (fallback to `torchinfo`)
import importlib, subprocess, sys


def ensure_package(pkg_name: str, import_name: str = None):
    name = import_name or pkg_name
    try:
        return importlib.import_module(name)
    except Exception:
        print(f"⚠️ {name} not found — installing {pkg_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg_name])
        importlib.invalidate_caches()
        module = importlib.import_module(name)
        print(f"✅ Installed and imported {name}")
        return module

# Primary: torchsummary
try:
    ts = ensure_package("torchsummary", "torchsummary")
    from torchsummary import summary
    print("✅ `torchsummary.summary` is available")
except Exception as e:
    print("⚠️ Failed to import `torchsummary` (will try fallback):", e)
    # Fallback: torchinfo
    try:
        ti = ensure_package("torchinfo", "torchinfo")
        from torchinfo import summary
        print("✅ Using fallback `torchinfo.summary`")
    except Exception as e2:
        print("❌ Failed to install fallback 'torchinfo':", e2)
        print("Please install `torchsummary` or `torchinfo` in your environment and re-run this cell.")

## 1. Import Libraries

In [ ]:
# Standard libraries
import os
import sys
import json
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any

# Data handling
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle

# Deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import spectral_norm
from torchsummary import summary

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

print("✅ Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Setup Paths and Configuration

In [ ]:
# Project paths
project_root = Path(r'd:\Projects\AI-Virtual-TryOn')
output_dir = project_root / 'outputs' / 'model_architecture'
output_dir.mkdir(parents=True, exist_ok=True)

# Load dataset configuration
with open(project_root / 'outputs' / 'dataset' / 'dataset_config.json', 'r') as f:
    dataset_config = json.load(f)

print(f"📁 Project Root: {project_root}")
print(f"📁 Output Directory: {output_dir}")
print(f"\n✅ Loaded dataset configuration")

# Device configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\n🖥️ Using device: {device}")

# Model configuration
model_config = {
    'input_channels': 41,  # 3 CA + 18 Pose + 20 Parsing
    'output_channels': 3,  # RGB
    'image_size': (1024, 768),
    'ngf': 64,  # Generator base channels
    'ndf': 64,  # Discriminator base channels
    'n_downsampling': 4,
    'n_blocks': 9,  # Residual blocks at bottleneck
    'use_dropout': True,
    'use_attention': True
}

print(f"\n📊 Model Configuration:")
for key, value in model_config.items():
    print(f"   {key}: {value}")

## 3. Building Blocks

Define reusable components for the networks.

In [ ]:
class ResidualBlock(nn.Module):
    """
    Residual block with two convolutional layers.
    """
    def __init__(self, channels: int, use_dropout: bool = False):
        super().__init__()
        
        layers = [
            nn.Conv2d(channels, channels, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True)
        ]
        
        if use_dropout:
            layers.append(nn.Dropout(0.5))
        
        layers += [
            nn.Conv2d(channels, channels, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm2d(channels)
        ]
        
        self.block = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x + self.block(x)


class SelfAttention(nn.Module):
    """
    Self-attention module for capturing long-range dependencies.
    """
    def __init__(self, channels: int):
        super().__init__()
        
        self.query = nn.Conv2d(channels, channels // 8, kernel_size=1)
        self.key = nn.Conv2d(channels, channels // 8, kernel_size=1)
        self.value = nn.Conv2d(channels, channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        batch, channels, height, width = x.size()
        
        # Query, Key, Value
        q = self.query(x).view(batch, -1, height * width).permute(0, 2, 1)
        k = self.key(x).view(batch, -1, height * width)
        v = self.value(x).view(batch, -1, height * width)
        
        # Attention map
        attention = F.softmax(torch.bmm(q, k), dim=-1)
        
        # Apply attention
        out = torch.bmm(v, attention.permute(0, 2, 1))
        out = out.view(batch, channels, height, width)
        
        return self.gamma * out + x


class DownsampleBlock(nn.Module):
    """
    Downsampling block for encoder.
    """
    def __init__(self, in_channels: int, out_channels: int, 
                 normalize: bool = True, dropout: float = 0.0):
        super().__init__()
        
        layers = [
            nn.Conv2d(in_channels, out_channels, kernel_size=4, 
                     stride=2, padding=1, bias=False)
        ]
        
        if normalize:
            layers.append(nn.InstanceNorm2d(out_channels))
        
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        
        if dropout > 0:
            layers.append(nn.Dropout(dropout))
        
        self.block = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.block(x)


class UpsampleBlock(nn.Module):
    """
    Upsampling block for decoder with skip connections.
    """
    def __init__(self, in_channels: int, out_channels: int, dropout: float = 0.0):
        super().__init__()
        
        layers = [
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4,
                              stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True)
        ]
        
        if dropout > 0:
            layers.append(nn.Dropout(dropout))
        
        self.block = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor, skip: Optional[torch.Tensor] = None) -> torch.Tensor:
        x = self.block(x)
        if skip is not None:
            x = torch.cat([x, skip], dim=1)
        return x


print("✅ Building blocks defined:")
print("   - ResidualBlock")
print("   - SelfAttention")
print("   - DownsampleBlock")
print("   - UpsampleBlock")

## 4. Generator Network

U-Net architecture with attention and residual connections.

In [ ]:
class Generator(nn.Module):
    """
    U-Net based Generator for Virtual Try-On.
    
    Input: [B, 41, H, W] (cloth-agnostic + pose + parsing)
    Output: [B, 3, H, W] (RGB image)
    """
    
    def __init__(self, 
                 in_channels: int = 41,
                 out_channels: int = 3,
                 ngf: int = 64,
                 n_downsampling: int = 4,
                 n_blocks: int = 9,
                 use_dropout: bool = True,
                 use_attention: bool = True):
        super().__init__()
        
        self.n_downsampling = n_downsampling
        
        # Initial convolution
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels, ngf, kernel_size=7, padding=3, bias=False),
            nn.InstanceNorm2d(ngf),
            nn.ReLU(inplace=True)
        )
        
        # Encoder (downsampling)
        self.encoder = nn.ModuleList()
        mult = 1
        for i in range(n_downsampling):
            mult_prev = mult
            mult = min(2 ** (i + 1), 8)
            self.encoder.append(
                DownsampleBlock(
                    ngf * mult_prev,
                    ngf * mult,
                    normalize=True,
                    dropout=0.0
                )
            )
        
        # Bottleneck (residual blocks)
        bottleneck_channels = ngf * mult
        self.bottleneck = nn.ModuleList()
        for _ in range(n_blocks):
            self.bottleneck.append(
                ResidualBlock(bottleneck_channels, use_dropout=use_dropout)
            )
        
        # Self-attention at bottleneck
        self.attention = SelfAttention(bottleneck_channels) if use_attention else None
        
        # Decoder (upsampling)
        self.decoder = nn.ModuleList()
        for i in range(n_downsampling):
            mult_prev = min(2 ** (n_downsampling - i), 8)
            mult = min(2 ** (n_downsampling - i - 1), 8)
            
            # Account for skip connections (double the input channels)
            in_ch = ngf * mult_prev
            out_ch = ngf * mult
            
            self.decoder.append(
                UpsampleBlock(
                    in_ch * 2,  # *2 for skip connection
                    out_ch,
                    dropout=0.5 if i < 3 and use_dropout else 0.0
                )
            )
        
        # Final convolution
        self.final = nn.Sequential(
            nn.Conv2d(ngf * 2, ngf, kernel_size=3, padding=1, bias=False),  # *2 for skip
            nn.InstanceNorm2d(ngf),
            nn.ReLU(inplace=True),
            nn.Conv2d(ngf, out_channels, kernel_size=7, padding=3),
            nn.Tanh()
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through Generator.
        
        Args:
            x: Input tensor [B, 41, H, W]
        
        Returns:
            Generated image [B, 3, H, W]
        """
        # Initial convolution
        x = self.initial(x)
        
        # Encoder with skip connections
        skip_connections = [x]
        for encoder_block in self.encoder:
            x = encoder_block(x)
            skip_connections.append(x)
        
        # Bottleneck
        for res_block in self.bottleneck:
            x = res_block(x)
        
        # Attention
        if self.attention is not None:
            x = self.attention(x)
        
        # Decoder with skip connections
        skip_connections = skip_connections[::-1]  # Reverse order
        for i, decoder_block in enumerate(self.decoder):
            skip = skip_connections[i]
            x = torch.cat([x, skip], dim=1)
            x = decoder_block(x)
        
        # Final convolution with last skip connection
        x = torch.cat([x, skip_connections[-1]], dim=1)
        x = self.final(x)
        
        return x


print("✅ Generator network defined")
print("   Architecture: U-Net with attention")
print("   Input: [B, 41, H, W]")
print("   Output: [B, 3, H, W]")

## 5. Discriminator Network

PatchGAN discriminator with spectral normalization.

In [ ]:
class Discriminator(nn.Module):
    """
    PatchGAN Discriminator for Virtual Try-On.
    
    Classifies 70x70 patches as real or fake.
    Uses spectral normalization for training stability.
    """
    
    def __init__(self,
                 in_channels: int = 6,  # 3 (image) + 3 (condition)
                 ndf: int = 64,
                 n_layers: int = 3,
                 use_spectral_norm: bool = True):
        super().__init__()
        
        norm_layer = spectral_norm if use_spectral_norm else lambda x: x
        
        # Initial layer (no normalization)
        layers = [
            norm_layer(nn.Conv2d(in_channels, ndf, kernel_size=4, stride=2, padding=1)),
            nn.LeakyReLU(0.2, inplace=True)
        ]
        
        # Intermediate layers
        mult = 1
        for n in range(1, n_layers):
            mult_prev = mult
            mult = min(2 ** n, 8)
            layers += [
                norm_layer(nn.Conv2d(ndf * mult_prev, ndf * mult, 
                                    kernel_size=4, stride=2, padding=1, bias=False)),
                nn.InstanceNorm2d(ndf * mult),
                nn.LeakyReLU(0.2, inplace=True)
            ]
        
        # Final layers
        mult_prev = mult
        mult = min(2 ** n_layers, 8)
        layers += [
            norm_layer(nn.Conv2d(ndf * mult_prev, ndf * mult,
                                kernel_size=4, stride=1, padding=1, bias=False)),
            nn.InstanceNorm2d(ndf * mult),
            nn.LeakyReLU(0.2, inplace=True)
        ]
        
        # Output layer
        layers.append(
            norm_layer(nn.Conv2d(ndf * mult, 1, kernel_size=4, stride=1, padding=1))
        )
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor, condition: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through Discriminator.
        
        Args:
            x: Input image [B, 3, H, W]
            condition: Condition image [B, 3, H, W] (e.g., cloth-agnostic)
        
        Returns:
            Patch predictions [B, 1, H', W']
        """
        # Concatenate image and condition
        x = torch.cat([x, condition], dim=1)
        return self.model(x)


print("✅ Discriminator network defined")
print("   Architecture: PatchGAN with spectral normalization")
print("   Input: [B, 6, H, W] (image + condition)")
print("   Output: [B, 1, H', W'] (patch predictions)")

## 6. Initialize Models

In [ ]:
# Initialize Generator
generator = Generator(
    in_channels=model_config['input_channels'],
    out_channels=model_config['output_channels'],
    ngf=model_config['ngf'],
    n_downsampling=model_config['n_downsampling'],
    n_blocks=model_config['n_blocks'],
    use_dropout=model_config['use_dropout'],
    use_attention=model_config['use_attention']
).to(device)

# Initialize Discriminator
discriminator = Discriminator(
    in_channels=6,  # RGB image + RGB condition
    ndf=model_config['ndf'],
    n_layers=3,
    use_spectral_norm=True
).to(device)

print("="*70)
print("🎯 MODELS INITIALIZED")
print("="*70)
print(f"\n✅ Generator created and moved to {device}")
print(f"✅ Discriminator created and moved to {device}")

## 7. Count Parameters

In [ ]:
def count_parameters(model: nn.Module) -> Tuple[int, int]:
    """
    Count total and trainable parameters.
    """
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable


# Count parameters
gen_total, gen_trainable = count_parameters(generator)
disc_total, disc_trainable = count_parameters(discriminator)

print("="*70)
print("📊 PARAMETER COUNT")
print("="*70)

print(f"\n🔷 Generator:")
print(f"   Total parameters: {gen_total:,}")
print(f"   Trainable parameters: {gen_trainable:,}")
print(f"   Memory: {gen_total * 4 / 1024**2:.2f} MB (float32)")

print(f"\n🔶 Discriminator:")
print(f"   Total parameters: {disc_total:,}")
print(f"   Trainable parameters: {disc_trainable:,}")
print(f"   Memory: {disc_total * 4 / 1024**2:.2f} MB (float32)")

print(f"\n📊 Total:")
print(f"   Combined parameters: {gen_total + disc_total:,}")
print(f"   Combined memory: {(gen_total + disc_total) * 4 / 1024**2:.2f} MB")

print("\n" + "="*70)

## 8. Test Forward Pass - Generator

In [ ]:
print("="*70)
print("🧪 TESTING GENERATOR FORWARD PASS")
print("="*70)

# Create dummy input
batch_size = 2
dummy_input = torch.randn(batch_size, 41, 1024, 768).to(device)

print(f"\n📥 Input shape: {tuple(dummy_input.shape)}")
print(f"   Channels breakdown:")
print(f"   - Cloth-agnostic RGB: 3")
print(f"   - Pose heatmaps: 18")
print(f"   - Parsing one-hot: 20")
print(f"   Total: 41 channels")

# Forward pass
generator.eval()
with torch.no_grad():
    output = generator(dummy_input)

print(f"\n📤 Output shape: {tuple(output.shape)}")
print(f"   Output range: [{output.min().item():.3f}, {output.max().item():.3f}]")
print(f"   Expected range: [-1, 1] (Tanh activation)")

# Check output
assert output.shape == (batch_size, 3, 1024, 768), "Output shape mismatch!"
assert output.min() >= -1.0 and output.max() <= 1.0, "Output range incorrect!"

print(f"\n✅ Generator forward pass successful!")
print(f"   Input: {tuple(dummy_input.shape)}")
print(f"   Output: {tuple(output.shape)}")

# Memory usage
if torch.cuda.is_available():
    mem_allocated = torch.cuda.memory_allocated() / 1024**2
    mem_reserved = torch.cuda.memory_reserved() / 1024**2
    print(f"\n💾 GPU Memory:")
    print(f"   Allocated: {mem_allocated:.2f} MB")
    print(f"   Reserved: {mem_reserved:.2f} MB")

print("\n" + "="*70)

## 9. Test Forward Pass - Discriminator

In [ ]:
print("="*70)
print("🧪 TESTING DISCRIMINATOR FORWARD PASS")
print("="*70)

# Create dummy inputs
dummy_image = torch.randn(batch_size, 3, 1024, 768).to(device)
dummy_condition = torch.randn(batch_size, 3, 1024, 768).to(device)

print(f"\n📥 Input shapes:")
print(f"   Image: {tuple(dummy_image.shape)}")
print(f"   Condition: {tuple(dummy_condition.shape)}")

# Forward pass
discriminator.eval()
with torch.no_grad():
    disc_output = discriminator(dummy_image, dummy_condition)

print(f"\n📤 Output shape: {tuple(disc_output.shape)}")
print(f"   Output range: [{disc_output.min().item():.3f}, {disc_output.max().item():.3f}]")

# Calculate receptive field size
patch_h, patch_w = disc_output.shape[2], disc_output.shape[3]
print(f"\n📊 Patch predictions:")
print(f"   Number of patches: {patch_h} × {patch_w} = {patch_h * patch_w}")
print(f"   Each patch covers ~70×70 pixels")

print(f"\n✅ Discriminator forward pass successful!")
print(f"   Image: {tuple(dummy_image.shape)}")
print(f"   Condition: {tuple(dummy_condition.shape)}")
print(f"   Output: {tuple(disc_output.shape)}")

print("\n" + "="*70)

## 10. Test Full Pipeline

In [ ]:
print("="*70)
print("🔄 TESTING FULL PIPELINE")
print("="*70)

# Create dummy input
print("\n📥 Creating dummy inputs...")
multi_channel_input = torch.randn(batch_size, 41, 1024, 768).to(device)
real_image = torch.randn(batch_size, 3, 1024, 768).to(device)
condition = torch.randn(batch_size, 3, 1024, 768).to(device)

# Generator forward pass
print("\n🔷 Generator: Generating fake images...")
generator.eval()
with torch.no_grad():
    fake_image = generator(multi_channel_input)

print(f"   Input: {tuple(multi_channel_input.shape)}")
print(f"   Output (fake): {tuple(fake_image.shape)}")

# Discriminator forward pass on fake
print("\n🔶 Discriminator: Evaluating fake images...")
discriminator.eval()
with torch.no_grad():
    pred_fake = discriminator(fake_image, condition)

print(f"   Fake prediction: {tuple(pred_fake.shape)}")
print(f"   Mean score: {pred_fake.mean().item():.3f}")

# Discriminator forward pass on real
print("\n🔶 Discriminator: Evaluating real images...")
with torch.no_grad():
    pred_real = discriminator(real_image, condition)

print(f"   Real prediction: {tuple(pred_real.shape)}")
print(f"   Mean score: {pred_real.mean().item():.3f}")

print("\n✅ Full pipeline test successful!")
print("\n📊 Pipeline Summary:")
print(f"   1. Multi-channel input [{batch_size}, 41, 1024, 768]")
print(f"   2. Generator → Fake image [{batch_size}, 3, 1024, 768]")
print(f"   3. Discriminator(fake) → Patches [{batch_size}, 1, {patch_h}, {patch_w}]")
print(f"   4. Discriminator(real) → Patches [{batch_size}, 1, {patch_h}, {patch_w}]")

print("\n" + "="*70)

## 11. Model Architecture Visualization

In [ ]:
def visualize_architecture():
    """
    Create visual diagram of model architecture.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 12))
    
    # Generator U-Net visualization
    ax1.set_title('Generator Architecture (U-Net)', fontsize=16, fontweight='bold', pad=20)
    ax1.axis('off')
    ax1.set_xlim(0, 10)
    ax1.set_ylim(0, 10)
    
    # Encoder path
    encoder_colors = ['#e8f4f8', '#b8dfe6', '#88cad4', '#58b5c2']
    encoder_positions = [(1, 8), (2, 7), (3, 6), (4, 5)]
    encoder_sizes = [(0.8, 0.8), (0.7, 0.7), (0.6, 0.6), (0.5, 0.5)]
    
    # Input
    ax1.add_patch(Rectangle((0.5, 8.5), 0.9, 0.9, 
                            facecolor='#c8e6c9', edgecolor='black', linewidth=2))
    ax1.text(0.95, 9.0, 'Input\n41ch', ha='center', va='center', fontsize=10, fontweight='bold')
    
    # Encoder blocks
    for i, (pos, size, color) in enumerate(zip(encoder_positions, encoder_sizes, encoder_colors)):
        channels = 64 * min(2**i, 8)
        ax1.add_patch(Rectangle((pos[0]-size[0]/2, pos[1]-size[1]/2), size[0], size[1],
                                facecolor=color, edgecolor='black', linewidth=2))
        ax1.text(pos[0], pos[1], f'Down\n{channels}ch', ha='center', va='center', 
                fontsize=9, fontweight='bold')
        # Arrow
        if i > 0:
            ax1.arrow(encoder_positions[i-1][0], encoder_positions[i-1][1]-0.5,
                     pos[0]-encoder_positions[i-1][0], pos[1]-encoder_positions[i-1][1]+0.5,
                     head_width=0.15, head_length=0.1, fc='black', ec='black')
    
    # Bottleneck
    ax1.add_patch(Rectangle((4.25, 4.25), 0.5, 0.5,
                            facecolor='#ff9999', edgecolor='black', linewidth=2))
    ax1.text(4.5, 4.5, 'Bottleneck\n512ch\n9 blocks\nAttention', 
            ha='center', va='center', fontsize=8, fontweight='bold')
    
    # Decoder path
    decoder_positions = [(6, 5), (7, 6), (8, 7), (9, 8)]
    decoder_sizes = [(0.5, 0.5), (0.6, 0.6), (0.7, 0.7), (0.8, 0.8)]
    
    for i, (pos, size, color) in enumerate(zip(decoder_positions, decoder_sizes, 
                                               encoder_colors[::-1])):
        channels = 64 * min(2**(3-i), 8)
        ax1.add_patch(Rectangle((pos[0]-size[0]/2, pos[1]-size[1]/2), size[0], size[1],
                                facecolor=color, edgecolor='black', linewidth=2))
        ax1.text(pos[0], pos[1], f'Up\n{channels}ch', ha='center', va='center',
                fontsize=9, fontweight='bold')
        # Arrow
        if i > 0:
            ax1.arrow(decoder_positions[i-1][0], decoder_positions[i-1][1]+0.5,
                     pos[0]-decoder_positions[i-1][0], pos[1]-decoder_positions[i-1][1]-0.5,
                     head_width=0.15, head_length=0.1, fc='black', ec='black')
    
    # Output
    ax1.add_patch(Rectangle((9.1, 8.5), 0.9, 0.9,
                            facecolor='#fff9c4', edgecolor='black', linewidth=2))
    ax1.text(9.55, 9.0, 'Output\n3ch', ha='center', va='center', fontsize=10, fontweight='bold')
    
    # Skip connections (dashed lines)
    for enc_pos, dec_pos in zip(encoder_positions, decoder_positions[::-1]):
        ax1.plot([enc_pos[0], dec_pos[0]], [enc_pos[1], dec_pos[1]],
                'r--', linewidth=1.5, alpha=0.6, label='Skip' if enc_pos == encoder_positions[0] else '')
    
    ax1.legend(loc='lower left', fontsize=10)
    
    # Discriminator visualization
    ax2.set_title('Discriminator Architecture (PatchGAN)', fontsize=16, fontweight='bold', pad=20)
    ax2.axis('off')
    ax2.set_xlim(0, 10)
    ax2.set_ylim(0, 10)
    
    # Input images
    ax2.add_patch(Rectangle((0.5, 8), 1.2, 1.5,
                            facecolor='#c8e6c9', edgecolor='black', linewidth=2))
    ax2.text(1.1, 8.75, 'Image\n3ch', ha='center', va='center', fontsize=10, fontweight='bold')
    
    ax2.add_patch(Rectangle((2.3, 8), 1.2, 1.5,
                            facecolor='#c8e6c9', edgecolor='black', linewidth=2))
    ax2.text(2.9, 8.75, 'Condition\n3ch', ha='center', va='center', fontsize=10, fontweight='bold')
    
    # Concatenate
    ax2.add_patch(Rectangle((4.3, 8), 1.2, 1.5,
                            facecolor='#ffe0b2', edgecolor='black', linewidth=2))
    ax2.text(4.9, 8.75, 'Concat\n6ch', ha='center', va='center', fontsize=10, fontweight='bold')
    
    # Conv layers
    disc_colors = ['#b8dfe6', '#88cad4', '#58b5c2', '#2891a0']
    disc_positions = [(2, 6), (4, 5), (6, 4), (8, 3)]
    disc_channels = [64, 128, 256, 512]
    
    for i, (pos, color, ch) in enumerate(zip(disc_positions, disc_colors, disc_channels)):
        size = 1.2 - i * 0.15
        ax2.add_patch(Rectangle((pos[0]-size/2, pos[1]-size/2), size, size,
                                facecolor=color, edgecolor='black', linewidth=2))
        ax2.text(pos[0], pos[1], f'Conv\n{ch}ch', ha='center', va='center',
                fontsize=9, fontweight='bold')
        # Arrow
        if i > 0:
            ax2.arrow(disc_positions[i-1][0], disc_positions[i-1][1]-0.7,
                     pos[0]-disc_positions[i-1][0], pos[1]-disc_positions[i-1][1]+0.7,
                     head_width=0.15, head_length=0.1, fc='black', ec='black')
    
    # Output
    ax2.add_patch(Rectangle((8.5, 1), 1.0, 1.0,
                            facecolor='#fff9c4', edgecolor='black', linewidth=2))
    ax2.text(9.0, 1.5, f'Patches\n{patch_h}×{patch_w}', 
            ha='center', va='center', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(output_dir / 'model_architecture.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Architecture visualization saved")


visualize_architecture()

## 12. Save Model Configuration

In [ ]:
# Create detailed model configuration
architecture_config = {
    'generator': {
        'type': 'U-Net',
        'input_channels': model_config['input_channels'],
        'output_channels': model_config['output_channels'],
        'base_channels': model_config['ngf'],
        'n_downsampling': model_config['n_downsampling'],
        'n_blocks': model_config['n_blocks'],
        'use_dropout': model_config['use_dropout'],
        'use_attention': model_config['use_attention'],
        'total_parameters': gen_total,
        'trainable_parameters': gen_trainable,
        'memory_mb': gen_total * 4 / 1024**2
    },
    'discriminator': {
        'type': 'PatchGAN',
        'input_channels': 6,
        'base_channels': model_config['ndf'],
        'n_layers': 3,
        'use_spectral_norm': True,
        'patch_size': '70x70',
        'output_patches': f'{patch_h}x{patch_w}',
        'total_parameters': disc_total,
        'trainable_parameters': disc_trainable,
        'memory_mb': disc_total * 4 / 1024**2
    },
    'total': {
        'combined_parameters': gen_total + disc_total,
        'combined_memory_mb': (gen_total + disc_total) * 4 / 1024**2
    },
    'image_size': model_config['image_size'],
    'device': device
}

# Save configuration
config_path = output_dir / 'model_architecture_config.json'
with open(config_path, 'w') as f:
    json.dump(architecture_config, f, indent=2)

print("="*70)
print("💾 MODEL CONFIGURATION SAVED")
print("="*70)

print(f"\n📄 Config saved to: {config_path}")

print(f"\n📊 Summary:")
print(f"   - Generator parameters: {gen_total:,}")
print(f"   - Discriminator parameters: {disc_total:,}")
print(f"   - Total parameters: {gen_total + disc_total:,}")
print(f"   - Total memory: {(gen_total + disc_total) * 4 / 1024**2:.2f} MB")

print("\n" + "="*70)

## 13. Summary

In [ ]:
print("="*70)
print("🎉 MODEL ARCHITECTURE COMPLETE!")
print("="*70)

print("\n✅ Completed Tasks:")
print("   1. ✓ Implemented building blocks (ResidualBlock, Attention, etc.)")
print("   2. ✓ Created Generator (U-Net with attention)")
print("   3. ✓ Created Discriminator (PatchGAN with spectral norm)")
print("   4. ✓ Initialized both models on GPU")
print("   5. ✓ Counted parameters and memory")
print("   6. ✓ Tested Generator forward pass")
print("   7. ✓ Tested Discriminator forward pass")
print("   8. ✓ Tested full pipeline")
print("   9. ✓ Visualized architecture")
print("   10. ✓ Saved model configuration")

print(f"\n📊 Model Statistics:")
print(f"   🔷 Generator:")
print(f"      - Architecture: U-Net with self-attention")
print(f"      - Input: [B, 41, 1024, 768]")
print(f"      - Output: [B, 3, 1024, 768]")
print(f"      - Parameters: {gen_total:,}")
print(f"      - Memory: {gen_total * 4 / 1024**2:.2f} MB")
print(f"   🔶 Discriminator:")
print(f"      - Architecture: PatchGAN with spectral normalization")
print(f"      - Input: [B, 6, 1024, 768] (image + condition)")
print(f"      - Output: [B, 1, {patch_h}, {patch_w}] (patch predictions)")
print(f"      - Parameters: {disc_total:,}")
print(f"      - Memory: {disc_total * 4 / 1024**2:.2f} MB")

print(f"\n📁 Generated Files:")
print(f"   - model_architecture.png")
print(f"   - model_architecture_config.json")

print("\n🚀 Ready for Next Steps:")
print("   1. Define loss functions (GAN, Perceptual, L1)")
print("   2. Implement training loop")
print("   3. Add checkpointing and logging")
print("   4. Start model training")

print("\n💡 Key Features:")
print("   - U-Net encoder-decoder architecture")
print("   - Skip connections for feature preservation")
print("   - Self-attention at bottleneck")
print("   - Residual blocks for stability")
print("   - PatchGAN discriminator (70×70)")
print("   - Spectral normalization for training stability")
print("   - Instance normalization throughout")

print("\n" + "="*70)

print("\n✅ MODEL ARCHITECTURE READY FOR TRAINING!")
print("="*70)

# Loss Functions


## 1. Import Libraries

In [ ]:
# Standard libraries
import os
import sys
import json
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any

# Data handling
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision.models import VGG19_Weights

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

print("✅ Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Setup Paths and Configuration

In [ ]:
# Project paths
project_root = Path(r'd:\Projects\AI-Virtual-TryOn')
output_dir = project_root / 'outputs' / 'loss_functions'
output_dir.mkdir(parents=True, exist_ok=True)

# Load model configuration
with open(project_root / 'outputs' / 'model_architecture' / 'model_architecture_config.json', 'r') as f:
    model_config = json.load(f)

print(f"📁 Project Root: {project_root}")
print(f"📁 Output Directory: {output_dir}")
print(f"\n✅ Loaded model configuration")

# Device configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\n🖥️ Using device: {device}")

# Loss weights configuration
loss_config = {
    'lambda_gan': 1.0,        # GAN loss weight
    'lambda_perceptual': 10.0, # Perceptual loss weight
    'lambda_l1': 10.0,        # L1 reconstruction weight
    'lambda_fm': 10.0,        # Feature matching weight
    'vgg_layers': ['relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'relu5_1']
}

print(f"\n📊 Loss Configuration:")
for key, value in loss_config.items():
    print(f"   {key}: {value}")

## 3. VGG19 Perceptual Loss Network

In [ ]:
class VGGPerceptualLoss(nn.Module):
    """
    Perceptual loss using VGG19 features.
    Computes L1 distance between features from multiple layers.
    """
    
    def __init__(self, 
                 layers: List[str] = ['relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'relu5_1'],
                 weights: Optional[List[float]] = None):
        super().__init__()
        
        # Load pretrained VGG19
        vgg = models.vgg19(weights=VGG19_Weights.IMAGENET1K_V1).features
        
        # Freeze VGG parameters
        for param in vgg.parameters():
            param.requires_grad = False
        
        # Layer mapping
        self.layer_name_mapping = {
            'relu1_1': '1',
            'relu1_2': '3',
            'relu2_1': '6',
            'relu2_2': '8',
            'relu3_1': '11',
            'relu3_2': '13',
            'relu3_3': '15',
            'relu3_4': '17',
            'relu4_1': '20',
            'relu4_2': '22',
            'relu4_3': '24',
            'relu4_4': '26',
            'relu5_1': '29',
            'relu5_2': '31',
            'relu5_3': '33',
            'relu5_4': '35',
        }
        
        # Build feature extractors
        self.features = nn.ModuleDict()
        for layer_name in layers:
            layer_idx = int(self.layer_name_mapping[layer_name])
            self.features[layer_name] = nn.Sequential(*[vgg[i] for i in range(layer_idx + 1)])
        
        # Layer weights (equal if not specified)
        self.weights = weights if weights is not None else [1.0] * len(layers)
        
        # Normalization for ImageNet
        self.register_buffer('mean', torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1))
        self.register_buffer('std', torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1))
    
    def normalize(self, x: torch.Tensor) -> torch.Tensor:
        """
        Normalize from [-1, 1] to ImageNet normalization.
        """
        # Convert from [-1, 1] to [0, 1]
        x = (x + 1) / 2
        # Apply ImageNet normalization
        x = (x - self.mean) / self.std
        return x
    
    def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        """
        Compute perceptual loss between x and y.
        
        Args:
            x: Generated image [B, 3, H, W]
            y: Target image [B, 3, H, W]
        
        Returns:
            Perceptual loss (scalar)
        """
        # Normalize inputs
        x = self.normalize(x)
        y = self.normalize(y)
        
        loss = 0.0
        
        # Extract features and compute loss
        for (layer_name, feature_extractor), weight in zip(self.features.items(), self.weights):
            x_feat = feature_extractor(x)
            y_feat = feature_extractor(y)
            
            # L1 distance between features
            loss += weight * F.l1_loss(x_feat, y_feat)
        
        return loss


print("✅ VGGPerceptualLoss defined")
print("   - Extracts features from VGG19")
print("   - Computes L1 distance across multiple layers")
print("   - Uses ImageNet pretrained weights")

## 4. GAN Loss (Adversarial Loss)

In [ ]:
class GANLoss(nn.Module):
    """
    GAN loss (adversarial loss).
    Supports multiple GAN objectives: vanilla, lsgan, hinge.
    """
    
    def __init__(self, gan_mode: str = 'lsgan', target_real_label: float = 1.0, 
                 target_fake_label: float = 0.0):
        """
        Args:
            gan_mode: Type of GAN loss ('vanilla', 'lsgan', 'hinge')
            target_real_label: Label for real images
            target_fake_label: Label for fake images
        """
        super().__init__()
        self.gan_mode = gan_mode
        self.real_label = target_real_label
        self.fake_label = target_fake_label
        
        if gan_mode == 'vanilla':
            self.loss = nn.BCEWithLogitsLoss()
        elif gan_mode == 'lsgan':
            self.loss = nn.MSELoss()
        elif gan_mode == 'hinge':
            # Hinge loss computed manually
            self.loss = None
        else:
            raise ValueError(f"Unsupported GAN mode: {gan_mode}")
    
    def get_target_tensor(self, prediction: torch.Tensor, target_is_real: bool) -> torch.Tensor:
        """
        Create target tensor with same size as prediction.
        """
        if target_is_real:
            target = torch.ones_like(prediction) * self.real_label
        else:
            target = torch.ones_like(prediction) * self.fake_label
        return target
    
    def forward(self, prediction: torch.Tensor, target_is_real: bool) -> torch.Tensor:
        """
        Compute GAN loss.
        
        Args:
            prediction: Discriminator output [B, 1, H, W]
            target_is_real: Whether target should be real (True) or fake (False)
        
        Returns:
            GAN loss (scalar)
        """
        if self.gan_mode == 'hinge':
            if target_is_real:
                # Discriminator loss for real: -min(0, -1 + D(x))
                loss = F.relu(1.0 - prediction).mean()
            else:
                # Discriminator loss for fake: -min(0, -1 - D(G(z)))
                loss = F.relu(1.0 + prediction).mean()
        else:
            target = self.get_target_tensor(prediction, target_is_real)
            loss = self.loss(prediction, target)
        
        return loss


print("✅ GANLoss defined")
print("   - Supports vanilla, LSGAN, and hinge loss")
print("   - Handles both real and fake predictions")

## 5. Feature Matching Loss

In [ ]:
class FeatureMatchingLoss(nn.Module):
    """
    Feature matching loss.
    Matches intermediate discriminator features between real and fake images.
    """
    
    def __init__(self):
        super().__init__()
    
    def forward(self, fake_features: List[torch.Tensor], 
                real_features: List[torch.Tensor]) -> torch.Tensor:
        """
        Compute feature matching loss.
        
        Args:
            fake_features: List of feature maps from discriminator (fake images)
            real_features: List of feature maps from discriminator (real images)
        
        Returns:
            Feature matching loss (scalar)
        """
        loss = 0.0
        
        for fake_feat, real_feat in zip(fake_features, real_features):
            loss += F.l1_loss(fake_feat, real_feat.detach())
        
        return loss / len(fake_features)


print("✅ FeatureMatchingLoss defined")
print("   - Matches discriminator intermediate features")
print("   - Helps stabilize GAN training")

## 6. Combined Loss Module

In [ ]:
class VITONLoss(nn.Module):
    """
    Combined loss for Virtual Try-On.
    Combines GAN, perceptual, L1, and feature matching losses.
    """
    
    def __init__(self, 
                 lambda_gan: float = 1.0,
                 lambda_perceptual: float = 10.0,
                 lambda_l1: float = 10.0,
                 lambda_fm: float = 10.0,
                 vgg_layers: List[str] = ['relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'relu5_1'],
                 gan_mode: str = 'lsgan'):
        super().__init__()
        
        # Loss weights
        self.lambda_gan = lambda_gan
        self.lambda_perceptual = lambda_perceptual
        self.lambda_l1 = lambda_l1
        self.lambda_fm = lambda_fm
        
        # Loss functions
        self.gan_loss = GANLoss(gan_mode=gan_mode)
        self.perceptual_loss = VGGPerceptualLoss(layers=vgg_layers)
        self.l1_loss = nn.L1Loss()
        self.fm_loss = FeatureMatchingLoss()
    
    def compute_generator_loss(self,
                              fake_image: torch.Tensor,
                              real_image: torch.Tensor,
                              disc_fake: torch.Tensor,
                              fake_features: Optional[List[torch.Tensor]] = None,
                              real_features: Optional[List[torch.Tensor]] = None) -> Dict[str, torch.Tensor]:
        """
        Compute generator losses.
        
        Args:
            fake_image: Generated image [B, 3, H, W]
            real_image: Target real image [B, 3, H, W]
            disc_fake: Discriminator output for fake image [B, 1, H', W']
            fake_features: Discriminator features for fake (optional)
            real_features: Discriminator features for real (optional)
        
        Returns:
            Dictionary with individual and total losses
        """
        losses = {}
        
        # GAN loss (fool discriminator)
        losses['gan'] = self.gan_loss(disc_fake, target_is_real=True) * self.lambda_gan
        
        # Perceptual loss
        losses['perceptual'] = self.perceptual_loss(fake_image, real_image) * self.lambda_perceptual
        
        # L1 reconstruction loss
        losses['l1'] = self.l1_loss(fake_image, real_image) * self.lambda_l1
        
        # Feature matching loss (if features provided)
        if fake_features is not None and real_features is not None:
            losses['fm'] = self.fm_loss(fake_features, real_features) * self.lambda_fm
        else:
            losses['fm'] = torch.tensor(0.0, device=fake_image.device)
        
        # Total loss
        losses['total'] = losses['gan'] + losses['perceptual'] + losses['l1'] + losses['fm']
        
        return losses
    
    def compute_discriminator_loss(self,
                                   disc_real: torch.Tensor,
                                   disc_fake: torch.Tensor) -> Dict[str, torch.Tensor]:
        """
        Compute discriminator losses.
        
        Args:
            disc_real: Discriminator output for real image [B, 1, H', W']
            disc_fake: Discriminator output for fake image [B, 1, H', W']
        
        Returns:
            Dictionary with individual and total losses
        """
        losses = {}
        
        # Real loss
        losses['real'] = self.gan_loss(disc_real, target_is_real=True)
        
        # Fake loss
        losses['fake'] = self.gan_loss(disc_fake, target_is_real=False)
        
        # Total loss (average of real and fake)
        losses['total'] = (losses['real'] + losses['fake']) * 0.5
        
        return losses


print("✅ VITONLoss defined")
print("   - Combines GAN, perceptual, L1, and feature matching losses")
print("   - Separate methods for generator and discriminator")
print("   - Returns detailed loss breakdown")

## 7. Initialize Loss Functions

In [ ]:
# Initialize combined loss
criterion = VITONLoss(
    lambda_gan=loss_config['lambda_gan'],
    lambda_perceptual=loss_config['lambda_perceptual'],
    lambda_l1=loss_config['lambda_l1'],
    lambda_fm=loss_config['lambda_fm'],
    vgg_layers=loss_config['vgg_layers'],
    gan_mode='lsgan'
).to(device)

print("="*70)
print("🎯 LOSS FUNCTIONS INITIALIZED")
print("="*70)
print(f"\n✅ VITONLoss created and moved to {device}")
print(f"\n📊 Loss weights:")
print(f"   - GAN: {loss_config['lambda_gan']}")
print(f"   - Perceptual: {loss_config['lambda_perceptual']}")
print(f"   - L1: {loss_config['lambda_l1']}")
print(f"   - Feature Matching: {loss_config['lambda_fm']}")

## 8. Test Loss Computations

In [ ]:
print("="*70)
print("🧪 TESTING LOSS COMPUTATIONS")
print("="*70)

# Create dummy data
batch_size = 2
fake_image = torch.randn(batch_size, 3, 1024, 768).to(device)
real_image = torch.randn(batch_size, 3, 1024, 768).to(device)
disc_fake = torch.randn(batch_size, 1, 126, 94).to(device)
disc_real = torch.randn(batch_size, 1, 126, 94).to(device)

print(f"\n📥 Input shapes:")
print(f"   Fake image: {tuple(fake_image.shape)}")
print(f"   Real image: {tuple(real_image.shape)}")
print(f"   Disc fake: {tuple(disc_fake.shape)}")
print(f"   Disc real: {tuple(disc_real.shape)}")

# Test generator loss
print(f"\n🔷 Testing Generator Loss...")
gen_losses = criterion.compute_generator_loss(
    fake_image=fake_image,
    real_image=real_image,
    disc_fake=disc_fake
)

print(f"\n📊 Generator Losses:")
for name, loss in gen_losses.items():
    print(f"   {name:15s}: {loss.item():.4f}")

# Test discriminator loss
print(f"\n🔶 Testing Discriminator Loss...")
disc_losses = criterion.compute_discriminator_loss(
    disc_real=disc_real,
    disc_fake=disc_fake
)

print(f"\n📊 Discriminator Losses:")
for name, loss in disc_losses.items():
    print(f"   {name:15s}: {loss.item():.4f}")

print("\n" + "="*70)
print("\n✅ Loss computation test successful!")
print("="*70)

## 9. Test Individual Loss Components

In [ ]:
print("="*70)
print("🔬 TESTING INDIVIDUAL LOSS COMPONENTS")
print("="*70)

# Test GAN loss
print("\n🔷 GAN Loss:")
gan_loss_real = criterion.gan_loss(disc_real, target_is_real=True)
gan_loss_fake = criterion.gan_loss(disc_fake, target_is_real=False)
print(f"   Real: {gan_loss_real.item():.4f}")
print(f"   Fake: {gan_loss_fake.item():.4f}")

# Test perceptual loss
print("\n🔷 Perceptual Loss:")
perceptual_loss = criterion.perceptual_loss(fake_image, real_image)
print(f"   Loss: {perceptual_loss.item():.4f}")

# Test L1 loss
print("\n🔷 L1 Loss:")
l1_loss = criterion.l1_loss(fake_image, real_image)
print(f"   Loss: {l1_loss.item():.4f}")

# Test with different similarity levels
print("\n🔬 Testing with different image similarities:")
identical = real_image.clone()
similar = real_image + torch.randn_like(real_image) * 0.1
different = torch.randn_like(real_image)

print("\n   Identical images:")
print(f"      Perceptual: {criterion.perceptual_loss(identical, real_image).item():.4f}")
print(f"      L1: {criterion.l1_loss(identical, real_image).item():.4f}")

print("\n   Similar images (noise=0.1):")
print(f"      Perceptual: {criterion.perceptual_loss(similar, real_image).item():.4f}")
print(f"      L1: {criterion.l1_loss(similar, real_image).item():.4f}")

print("\n   Different images (random):")
print(f"      Perceptual: {criterion.perceptual_loss(different, real_image).item():.4f}")
print(f"      L1: {criterion.l1_loss(different, real_image).item():.4f}")

print("\n" + "="*70)
print("\n✅ Individual loss tests successful!")
print("="*70)

## 10. Visualize Loss Landscape

In [ ]:
def visualize_loss_comparison():
    """
    Visualize how different losses respond to image similarity.
    """
    # Create images with varying levels of similarity
    base_image = torch.randn(1, 3, 256, 256).to(device)
    noise_levels = np.linspace(0, 2, 20)
    
    perceptual_losses = []
    l1_losses = []
    
    for noise in noise_levels:
        noisy_image = base_image + torch.randn_like(base_image) * noise
        noisy_image = torch.clamp(noisy_image, -1, 1)
        
        with torch.no_grad():
            perceptual = criterion.perceptual_loss(noisy_image, base_image).item()
            l1 = criterion.l1_loss(noisy_image, base_image).item()
        
        perceptual_losses.append(perceptual)
        l1_losses.append(l1)
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss curves
    ax1.plot(noise_levels, perceptual_losses, 'b-', linewidth=2, label='Perceptual Loss')
    ax1.plot(noise_levels, l1_losses, 'r-', linewidth=2, label='L1 Loss')
    ax1.set_xlabel('Noise Level', fontsize=12)
    ax1.set_ylabel('Loss Value', fontsize=12)
    ax1.set_title('Loss vs Image Similarity', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # Normalized comparison
    perceptual_norm = np.array(perceptual_losses) / max(perceptual_losses)
    l1_norm = np.array(l1_losses) / max(l1_losses)
    
    ax2.plot(noise_levels, perceptual_norm, 'b-', linewidth=2, label='Perceptual (normalized)')
    ax2.plot(noise_levels, l1_norm, 'r-', linewidth=2, label='L1 (normalized)')
    ax2.set_xlabel('Noise Level', fontsize=12)
    ax2.set_ylabel('Normalized Loss', fontsize=12)
    ax2.set_title('Normalized Loss Comparison', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(output_dir / 'loss_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Loss comparison visualization saved")


visualize_loss_comparison()

## 11. Loss Weights Sensitivity Analysis

In [ ]:
def analyze_loss_weights():
    """
    Analyze the impact of different loss weights.
    """
    # Sample data
    fake = torch.randn(1, 3, 256, 256).to(device)
    real = torch.randn(1, 3, 256, 256).to(device)
    disc = torch.randn(1, 1, 31, 23).to(device)
    
    # Test different weight configurations
    weight_configs = [
        {'lambda_gan': 1, 'lambda_perceptual': 0, 'lambda_l1': 0, 'lambda_fm': 0},
        {'lambda_gan': 0, 'lambda_perceptual': 1, 'lambda_l1': 0, 'lambda_fm': 0},
        {'lambda_gan': 0, 'lambda_perceptual': 0, 'lambda_l1': 1, 'lambda_fm': 0},
        {'lambda_gan': 1, 'lambda_perceptual': 10, 'lambda_l1': 10, 'lambda_fm': 10},
    ]
    
    config_names = ['GAN only', 'Perceptual only', 'L1 only', 'Combined (balanced)']
    
    results = []
    
    for config, name in zip(weight_configs, config_names):
        test_criterion = VITONLoss(**config, vgg_layers=loss_config['vgg_layers']).to(device)
        
        with torch.no_grad():
            losses = test_criterion.compute_generator_loss(fake, real, disc)
        
        results.append({
            'config': name,
            'total': losses['total'].item(),
            'gan': losses['gan'].item(),
            'perceptual': losses['perceptual'].item(),
            'l1': losses['l1'].item(),
            'fm': losses['fm'].item()
        })
    
    # Create DataFrame
    df = pd.DataFrame(results)
    
    print("="*70)
    print("📊 LOSS WEIGHTS SENSITIVITY ANALYSIS")
    print("="*70)
    print(f"\n{df.to_string(index=False)}")
    
    # Visualize
    fig, ax = plt.subplots(figsize=(12, 6))
    
    x = np.arange(len(config_names))
    width = 0.15
    
    ax.bar(x - 2*width, df['gan'], width, label='GAN', color='#ff9999')
    ax.bar(x - width, df['perceptual'], width, label='Perceptual', color='#66b3ff')
    ax.bar(x, df['l1'], width, label='L1', color='#99ff99')
    ax.bar(x + width, df['fm'], width, label='Feature Matching', color='#ffcc99')
    ax.bar(x + 2*width, df['total'], width, label='Total', color='#ff99cc', alpha=0.7)
    
    ax.set_xlabel('Configuration', fontsize=12, fontweight='bold')
    ax.set_ylabel('Loss Value', fontsize=12, fontweight='bold')
    ax.set_title('Loss Components Across Different Weight Configurations', 
                fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(config_names, rotation=15, ha='right')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(output_dir / 'loss_weights_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Loss weights analysis complete")


analyze_loss_weights()

## 12. Save Loss Configuration

In [ ]:
# Create detailed loss configuration
loss_full_config = {
    'loss_weights': {
        'lambda_gan': loss_config['lambda_gan'],
        'lambda_perceptual': loss_config['lambda_perceptual'],
        'lambda_l1': loss_config['lambda_l1'],
        'lambda_fm': loss_config['lambda_fm']
    },
    'gan_loss': {
        'type': 'lsgan',
        'target_real': 1.0,
        'target_fake': 0.0
    },
    'perceptual_loss': {
        'network': 'VGG19',
        'layers': loss_config['vgg_layers'],
        'pretrained': 'ImageNet'
    },
    'reconstruction_loss': {
        'type': 'L1',
        'pixel_wise': True
    },
    'feature_matching': {
        'enabled': True,
        'discriminator_features': True
    },
    'device': device
}

# Save configuration
config_path = output_dir / 'loss_config.json'
with open(config_path, 'w') as f:
    json.dump(loss_full_config, f, indent=2)

print("="*70)
print("💾 LOSS CONFIGURATION SAVED")
print("="*70)

print(f"\n📄 Config saved to: {config_path}")

print(f"\n📊 Summary:")
print(f"   - GAN weight: {loss_config['lambda_gan']}")
print(f"   - Perceptual weight: {loss_config['lambda_perceptual']}")
print(f"   - L1 weight: {loss_config['lambda_l1']}")
print(f"   - Feature matching weight: {loss_config['lambda_fm']}")
print(f"   - VGG layers: {len(loss_config['vgg_layers'])}")

print("\n" + "="*70)

## 13. Summary

In [ ]:
print("="*70)
print("🎉 LOSS FUNCTIONS COMPLETE!")
print("="*70)

print("\n✅ Completed Tasks:")
print("   1. ✓ Implemented VGG19 perceptual loss")
print("   2. ✓ Implemented GAN loss (LSGAN)")
print("   3. ✓ Implemented feature matching loss")
print("   4. ✓ Created combined VITONLoss module")
print("   5. ✓ Tested all loss computations")
print("   6. ✓ Tested individual loss components")
print("   7. ✓ Visualized loss comparisons")
print("   8. ✓ Analyzed loss weight sensitivity")
print("   9. ✓ Saved loss configuration")

print(f"\n📊 Loss Components:")
print(f"   🔷 GAN Loss:")
print(f"      - Type: LSGAN (Mean Squared Error)")
print(f"      - Weight: {loss_config['lambda_gan']}")
print(f"      - Purpose: Adversarial training for realistic generation")
print(f"   🔷 Perceptual Loss:")
print(f"      - Network: VGG19 (ImageNet pretrained)")
print(f"      - Layers: {len(loss_config['vgg_layers'])} layers")
print(f"      - Weight: {loss_config['lambda_perceptual']}")
print(f"      - Purpose: Feature-level similarity")
print(f"   🔷 L1 Loss:")
print(f"      - Type: Mean Absolute Error")
print(f"      - Weight: {loss_config['lambda_l1']}")
print(f"      - Purpose: Pixel-wise reconstruction")
print(f"   🔷 Feature Matching:")
print(f"      - Type: Discriminator feature matching")
print(f"      - Weight: {loss_config['lambda_fm']}")
print(f"      - Purpose: Training stability")

print(f"\n📁 Generated Files:")
print(f"   - loss_comparison.png")
print(f"   - loss_weights_analysis.png")
print(f"   - loss_config.json")

print("\n🚀 Ready for Next Steps:")
print("   1. Implement training loop")
print("   2. Add checkpointing and logging")
print("   3. Implement learning rate scheduling")
print("   4. Start model training")

print("\n💡 Key Features:")
print("   - Multi-scale perceptual loss (5 VGG layers)")
print("   - Stable GAN training with LSGAN")
print("   - Feature matching for better convergence")
print("   - Balanced loss weights for quality")
print("   - Comprehensive loss breakdown for monitoring")

print("\n" + "="*70)

print("\n✅ LOSS FUNCTIONS READY FOR TRAINING!")
print("="*70)